# Calculate Shortest Path and Generate OD Matrx
This notebook 
* attribute farms and potential digester locations to road network nodes
* calculate all-pair shortest paths between farms and all potential digester locations
* save the OD matrix (C_ij) as dictionary {p0: {f0: distance, f1: distance}...}

In [10]:
from cflp_function import store_data_to_pickle
# import matplotlib.pyplot as plt
import networkx as nx
import osmnx as ox
import geopandas as gpd
import pandas as pd
import h3
from shapely.ops import nearest_points
from shapely.geometry import Polygon, Point
import pydeck as pdk

In [2]:
# Read in the df of location of interests from suitability analysis
loi = pd.read_csv('./hex/loi.csv')

In [3]:
# Define a function to convert H3 index to Shapely polygon and center coordinates
def cell_to_shapely_polygon(h3_index):
    # hex_center_coords = h3.h3_to_geo(h3_index)
    coords = h3.h3_to_geo_boundary(h3_index)
    flipped = tuple(coord[::-1] for coord in coords)
    # center_point = Point(hex_center_coords)
    return Polygon(flipped) #, center_point

In [4]:
def cell_to_shaply_point(h3_index):
    lat, lon = h3.h3_to_geo(h3_index)
    return Point(lon, lat)

In [5]:
def loi_to_gdf(loi):
    loi['geometry'] = loi['hex9'].apply(cell_to_shaply_point) # can change the function here
    loi_gdf = gpd.GeoDataFrame(loi, geometry='geometry', crs=4326)
    return loi_gdf

In [7]:
# loi['polygon'] = loi['hex9'].apply(lambda x: cell_to_shapely_polygon(x)) 

In [8]:
loi['geometry'] = loi['hex9'].apply(cell_to_shaply_point)

In [9]:
# Convert to GeoDataFrame
loi_gdf = gpd.GeoDataFrame(loi, geometry='geometry', crs=4326)

,Unnamed: 0,hex9,color,fuzzy,geometry
0,493,891f160150fffff,"[68, 2, 85]",0.990388,POINT (6.99370 52.31514)
1,723,891f1602237ffff,"[68, 1, 84]",0.991347,POINT (7.01376 52.37152)
2,2056,891f160a83bffff,"[68, 1, 84]",0.994438,POINT (7.01006 52.24823)
3,4702,891f16196a3ffff,"[68, 1, 84]",0.990700,POINT (6.71446 52.27222)
4,7899,891f1656b2fffff,"[68, 2, 85]",0.990355,POINT (6.80040 52.20379)
5,10715,891f16c5a97ffff,"[68, 1, 84]",0.993799,POINT (6.67223 52.35803)
6,10857,891f16c5e13ffff,"[68, 2, 85]",0.990121,POINT (6.65118 52.36532)


In [12]:
loi_gdf['lat'] = loi_gdf['geometry'].y
loi_gdf['lon'] = loi_gdf['geometry'].x

In [14]:
layer = pdk.Layer(
        type='ScatterplotLayer',
        data=loi_gdf,
        get_position='[lon, lat]',
        get_radius=800,
        get_fill_color='color',
        pickable=True,
        auto_highlight=True, 
        get_line_color=[255, 255, 255],
        get_line_width=2,
    )

# Set the viewport location
view_state = pdk.ViewState(longitude=6.747489560596507, latitude=52.316862707395394, zoom=9, bearing=0, pitch=0)

# Render
r = pdk.Deck(layers=[layer], initial_view_state=view_state, tooltip={"text": "Count: {value}"})
r.to_html("h3_hexagon_layer.html")

In [10]:
main_crs ='EPSG:4326'

## 1. Farms to Nodes (QGIS)
Inputs:
* n - road network nodes (GeoDataFrame)
* farm - farm location points (GeoDataFrame)
* polygon_gdf - potential digester location polygons (GeoDataFrame)

In [11]:
n = gpd.read_file(r"./osm_network/G_n.shp")
n.head()

,osmid,y,x,lon,lat,highway,geometry
0,0,6.863327e+06,732329.118328,6.578624,52.348177,None,POINT (732329.118 6863326.894)
1,1,6.862993e+06,732029.301544,6.575931,52.346343,None,POINT (732029.302 6862992.539)
2,2,6.863769e+06,732826.404757,NaN,NaN,None,POINT (732826.405 6863768.536)
3,3,6.863381e+06,733531.413357,6.589425,52.348473,None,POINT (733531.413 6863380.689)
4,4,6.865492e+06,731879.617087,NaN,NaN,None,POINT (731879.617 6865492.254)


In [ ]:
farm = gpd.read_file("./farm/farm_new.shp")

In [14]:
n = n.to_crs(main_crs)

In [17]:
def find_closest_osmid(gdf, n):
    gdf['closest_osmid'] = gdf['geometry'].apply(
        lambda location: n.loc[n['geometry'] == nearest_points(location, n.unary_union)[1], 'osmid'].iloc[0])

In [32]:
find_closest_osmid(farm, n)
find_closest_osmid(loi_gdf, n)

## 2. Calculate OD Matrix
Inputs:
* Road network with farm information at nodes 
* Set of potential digester locations 

In [29]:
g = ox.load_graphml('./osm_network/G.graphml') 

In [30]:
orig = farm['closest_osmid'].unique().tolist()
dest = loi_gdf['closest_osmid'].unique().tolist()

[528, 845, 655, 855, 195, 226, 81, 643, 888, 739, 249, 105, 144, 96, 222, 92, 67, 787, 165, 321, 615, 354, 340, 98, 257, 383, 630, 727, 564, 578, 84, 93, 347, 170, 486, 117, 333, 54, 150, 11, 874, 597, 208, 94, 338, 570, 241, 410, 701, 614, 745, 654, 679, 673, 741, 732, 502, 771, 814, 9, 263, 229, 76, 286, 210, 942, 64, 17, 23, 16, 203, 319, 100, 198, 459, 824, 791, 815, 725, 764, 959, 961, 828, 861, 599, 457, 164, 836, 310, 318, 682, 698, 50, 998, 331, 413, 442, 474, 290, 213, 473, 505]


In [35]:
# Initialize an empty OD matrix
od_matrix = {}
# Calculate shortest path between all pair orig (farm) and dest (potential location)
for origin in orig:
    od_matrix[origin] = {}
    for destination in dest:
        distance = nx.shortest_path_length(g, origin, destination, weight='length')
        od_matrix[origin][destination] = distance/1000 # convert from m to km
# {orig:{dest:distance, dest:distance....}}

In [36]:
# Initialize an empty nested dictionary
new_nested_dict = {}

# Create a new nested dictionary with DataFrame index as keys
for idx, row in farm.iterrows():
    osmid_value = row['closest_osmid']
    if osmid_value in od_matrix:
        new_nested_dict[idx] = od_matrix[osmid_value]

# {farm1:{dest:distance, dest:distance....}} # some nodes are the closest for more than 1 farms, so now we make sure the dictionary is with key of all farms and each take the 
# associated distances (i.e. some farms will have the same od to all dest)

In [38]:
placeholders = {i:j for i, j in zip(loi_gdf.index.values, loi_gdf['closest_osmid'])}

In [39]:
restructured_od = {}

for farm, distances in new_nested_dict.items():
    restructured_od[farm] = {}
    for index, placeholder in placeholders.items():
        restructured_od[farm][index] = distances.get(placeholder, None)

transport_cost = {(farm, index): distance for farm, distances in restructured_od.items() for index, distance in distances.items()}

In [40]:
# Convert from distance to cost
cost_per_km = 0.69
transport_cost = {key: value * cost_per_km for key, value in transport_cost.items()}

In [41]:
for key, value in transport_cost.items():
    print(f"{key}: {value}")

(0, 0): 20.879368989588755
(0, 1): 22.11067561961571
(0, 2): 24.76996887397212
(0, 3): 10.331400912299163
(0, 4): 16.48783157902148
(0, 5): 9.765524326118536
(0, 6): 12.31285211857259
(1, 0): 23.48315166507738
(1, 1): 15.782359498867933
(1, 2): 29.38886133287685
(1, 3): 28.07158567835628
(1, 4): 32.51483208041656
(1, 5): 17.665240410839914
(1, 6): 18.18472818218271
(2, 0): 15.636596974304515
(2, 1): 16.478363168985226
(2, 2): 21.54230664210398
(2, 3): 20.225030987583406
(2, 4): 24.668277389643688
(2, 5): 17.719491058664108
(2, 6): 18.238978830006904
(3, 0): 25.670549897271908
(3, 1): 22.775517637417718
(3, 2): 31.576259565071375
(3, 3): 24.90090565458016
(3, 4): 32.5761241755121
(3, 5): 12.842123481541995
(3, 6): 13.361611252884794
(4, 0): 37.77534415695811
(4, 1): 43.26062605207588
(4, 2): 39.41741583933663
(4, 3): 15.237713383792704
(4, 4): 25.507989364140474
(4, 5): 25.922910644476676
(4, 6): 25.015444231533962
(5, 0): 35.83576035764303
(5, 1): 41.3210422527608
(5, 2): 37.4778320400

In [114]:
# # Print the new nested dictionary
# print("New Nested Dictionary:")
# for key, value in transportation_cost.items():
#     print(f"{key}: {value}")

In [49]:
store_data_to_pickle(transport_cost, 'app_data', 'c_test.pickle')
store_data_to_pickle(loi_gdf.index.tolist(), 'app_data', 'Plant_test_2.pickle')